# 参考文献

- [1] https://mp.weixin.qq.com/s/RPVzp2gOvufMt2SymxGMVw

# 目录

- [确定数据](#确定数据)
- [是否使用预训练模型](#是否使用预训练模型)
- [归一化层（normalization-layers）](#归一化层（normalization-layers）)

# 确定数据



- 确保输入(x)对应于一个label(y)，多分类任务有多个label
- 确保预测数据中label与训练label一致，例如one-hot格式，cate格式，数值范围等。之前首焦模型中就用feat作为label导致取值范围不同都默认为负样本，训练失败。

# 是否使用预训练模型

- 如果问题域中的数据集类似于ImageNet数据集，可以使用预训练模型，如VGG，Resnet，DenseNet，Xception等。建议从较少的网络层开始尝试（例如VGG-16, ResNet-50或densen -121层），获得最佳参数后再尝试同种网络的更多的网络层。

- 也可以只使用初始的几层（锁定不更新）提取基本特征（例如锯齿、形状等）高阶层次自定义。

- 微调几层（预训练模型中浅层锁定，高层更新），或者数据集较小，只训练最后的分类层。还可以在需要微调（更新）的conv层后加入dropout层，抵抗过拟合。但是**dropout在层次少的模型（数据少）中效果好，高层的（数据多，用了bn就不用dropout了）效果一般**。

- 如果数据集和预训练网络的数据集（ImageNet）不相似，则从头构建并训练模型。

# 归一化层（normalization-layers）

- 较大`batch size`（>10），使用**批标准化层（BatchNormalization）**
- 较小`batch size`（= 1），使用**InstanceNormalization**
- 较小bs使用BN层性能会降低
- layernorm？
- groupnorm？

# 防止过拟合

模型参数数量 > 训练数据数量 ->产生过拟合

防止过拟合：

1. 数据增强
2. L1正则化
3. L2正则化
4. Dropout
5. DropoutConnet
6. early stopping
7. 增加训练数据（推荐算法常用）

## 数据增强

提升算法性能，增加数据数量。

- 给训练数据添加转换、扰动增加训练数据，包括：水平、垂直翻转，剪裁，色彩变换，扩展，旋转等

## 正则化



 ### L1 和 L2 正则化
 
 - L1正则化：向loss函数中添加可训练参数绝对值总和
 
     - 在非常稀疏的向量上计算效率很低
     - 可以获得稀疏模型，使得大部分参数稀疏，会把不重要的特征直接置零（可减少存储空间）
     - loss的原点不可导
     - 解不唯一
     - 特征筛选器
 
 - L2正则化：向loss函数中添加可训练参数的平方综合
     - 唯一解
     - 计算方便，实现简单
     - 几乎不会把参数置零
 

### Dropout

Bagging 是通过**结合多个模型降低泛化误差**的技术，主要的做法是**分别训练几个不同的模型，然后让所有模型表决测试样例的输出**。而 Dropout 可以被认为是**集成了大量深层神经网络的 Bagging 方法**，因此它提供了一种*廉价的 Bagging 集成近似方法*，能够训练和评估的神经网络。

Dropout 指**暂时丢弃一部分神经元及其连接**。随机丢弃神经元可以防止过拟合，同时**指数级、高效地连接不同网络架构**。神经元被丢弃的概率为 1 − p，**减少神经元之间的共适应**。隐藏层通常以 0.5 的概率丢弃神经元。使用完整网络（每个节点的输出权重为 p）对所有 2^n 个 dropout 神经元的样本平均值进行近似计算。Dropout 显著降低了过拟合，同时通过避免在训练数据上的训练节点提高了算法的学习速度。

- 增加训练时间

### Drop Connect

是dropout的一般化。

drop connect将网络架构权重的一个随机选择自己置零，而dropout是对每层的激活值的子集置零。

- 对于每个单元，接收的输入数据都是上一层输出的子集，所以两者都有泛化性能。
- 二者在模型中引入稀疏性，dropout是输出向量稀疏性，drop connect是权重稀疏性。



### early stopping

限制模型最小化loss的训练迭代次数，防止训练中过度表达的模型泛化性能差。

epoch少 -> 欠拟合（方差较小，偏差较大）

epoch大 -> 过拟合（方差较大，偏差较小）

设置好停止条件即可。

### SpatialDropout

- 用于较低层而不是较高层
- 两个+conv层对相同的输入进行计算，得到的feature map后使用spatial dropout。

因为多个卷积层从相同的输入提取的feature map可能是相关的，所以用spatial drop删除相关特征，防止网络过拟合。

![spatialdropout](../imgs/spatialdropout.jpg)

# 捕捉目标上下文信息

使用多尺度pooling

# 数据不平衡

- 过采样
- 欠采样
- 类别加权
- 使用其他评估指标：
    - precision：类别精度，当模型判断一个样本属于该类的情况下，判断结果的可信度
    - recall：类别召回率，模型能够检测到该类的比率。
    - F1：precision和recall的调和平均值，将一个类的precision和recall结合在一个指标中。
    
    - 高precision + 高recall：模型能够很好的检测该类
    - 高precision + 低recall：模型不能很好的检测该类，但是判断结果是高度可信的
    - 低precision + 高recall：模型能很好的检测该类，但是判断结果中包含其他类的样本
    - 低precision + 低recall：模型不能很好的检测该类

# 测试网络容量

为了确定你的网络容量，尝试用一小部分训练例子来超载你的网络(andrej karpathy的提示)。如果它没有超载，增加你的网络容量。在过拟合后，使用正则化技巧如L1、L2、Dropout或其他技术来对抗过拟合。

- L1

- L2

# 优化器